In [ ]:
!git clone https://github.com/Karandh1r/TextMiningAssignment-1.git

Cloning into 'TextMiningAssignment-1'...
remote: Enumerating objects: 244, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 244 (delta 86), reused 217 (delta 59), pack-reused 0
Receiving objects: 100% (244/244), 16.72 MiB | 15.02 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [ ]:
!pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=a360506cd999f59d174556374df50ff31495f18327cc2778a5f9ceddb216271f
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=04f5fa3f3d071076553e3bb75e91718369b2b2cf5be69246255becc0691847aa
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyldavis sklearn


In [1]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/karandhir/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
from sklearn import preprocessing
from sklearn.cluster import KMeans
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
import pyLDAvis
import pyLDAvis.sklearn   
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
  

In [6]:
le = LabelEncoder()
ohe = OneHotEncoder()

file_name_movies = 'moviedata.csv'
file_movie_reviews = 'reviewdata.csv'

df_movie_reviews = pd.read_csv(file_movie_reviews)
print(df_movie_reviews.columns)

Index(['MovieId', 'UserReviews'], dtype='object')


In [7]:
def lemmatize_words(text):
        lemmatizer = WordNetLemmatizer()
        words = text.split()
        words = [lemmatizer.lemmatize(word) for word in words]
        return ' '.join(words)

def stemming_words(text):
        ps = PorterStemmer()
        words = text.split()
        words = [ps.stem(word) for word in words]
        return ' '.join(words) 

df_movie_reviews['UserReviews'] = df_movie_reviews['UserReviews'].apply(lemmatize_words)
df_movie_reviews['UserReviews'] = df_movie_reviews['UserReviews'].apply(stemming_words) 

vectorizer = CountVectorizer(stop_words='english',max_features=50)
X = vectorizer.fit_transform(df_movie_reviews['UserReviews'])
column_names = vectorizer.get_feature_names_out()
count_vec_columns = column_names
vectorized_df = pd.DataFrame(X.toarray(),columns=column_names)
num_topics = 7
lda_model_DH = LatentDirichletAllocation(n_components=num_topics, 
                                         max_iter=100, learning_method='online')
LDA_DH_Model = lda_model_DH.fit_transform(vectorized_df)
print("SIZE: ", LDA_DH_Model.shape) 

def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic:  ", idx)  
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
print_topics(lda_model_DH, vectorizer, 15)
word_topic = np.array(lda_model_DH.components_)
word_topic = word_topic.transpose()
num_top_words = 15
vocab_array = np.asarray(column_names)
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model_DH, X, vectorizer)


SIZE:  (879, 7)
Topic:   0
[('thi', 316.0019881673617), ('movi', 300.1560019899698), ('hi', 161.3230720594955), ('make', 112.33348458479885), ('look', 78.44232797913098), ('just', 71.999579444168), ('watch', 64.72187101373326), ('good', 59.25860202691187), ('movie', 56.70241380431272), ('time', 49.14549118385743), ('cast', 46.26472621567944), ('think', 42.1702913831468), ('becaus', 41.669515753502324), ('way', 37.37560417468717), ('work', 36.51767108870987)]
Topic:   1
[('like', 154.372498298507), ('woman', 100.53057503531772), ('ha', 100.11856884910534), ('thi', 84.43063258434148), ('stori', 83.22830436600921), ('peopl', 82.41645487368781), ('feel', 81.54195354934991), ('mani', 72.25072940135419), ('life', 60.851705102374275), ('just', 59.59067135521595), ('love', 59.232348335397965), ('charact', 56.742049356439), ('don', 54.09751460889715), ('thing', 49.32474242614556), ('think', 45.68070347793606)]
Topic:   2
[('thi', 259.2982483016654), ('movi', 218.70549058420582), ('bad', 177.021

/Users/karandhir/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/karandhir/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LatentDirichletAllocation was fitted with feature names
  warnings.warn(
/Users/karandhir/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/karandhir/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.128716  0.046152       1        1  19.654301
2      0.113907  0.079771       2        1  19.373008
1      0.059410  0.046998       3        1  18.512816
6      0.090394  0.040779       4        1  16.338214
4      0.142794 -0.010009       5        1  15.529758
5     -0.142862 -0.400790       6        1   6.869558
3     -0.392359  0.197099       7        1   3.722345, topic_info=      Term        Freq       Total Category  logprob  loglift
44      wa  684.000000  684.000000  Default  30.0000  30.0000
13    film  528.000000  528.000000  Default  29.0000  29.0000
16   great  161.000000  161.000000  Default  28.0000  28.0000
4     best  124.000000  124.000000  Default  27.0000  27.0000
29    movi  541.000000  541.000000  Default  26.0000  26.0000
..     ...         ...         ...      ...      ...      ...
39     thi    0.179474  965.591055   Topic7  -7.5190  -5.2996
42    time    0.179471  166.199591   Topic7  -7.5190  -3.5401
35  realli    0.179423  150.647939   Topic7  -7.5193  -3.4421
16   great    0.179409  161.069193   Topic7  -7.5194  -3.5091
13    film    0.179397  528.863971   Topic7  -7.5195  -4.6981

[271 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         1  0.299667    act
0         2  0.155383    act
0         3  0.377359    act
0         4  0.011099    act
0         5  0.166482    act
...     ...       ...    ...
48        4  0.180948  woman
49        1  0.461699   work
49        3  0.397574   work
49        5  0.012825   work
49        7  0.128250   work

[181 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 7, 5, 6, 4])